In [ ]:
%reset
%load_ext autoreload
%autoreload 2

import pickle
import math

# The deformation module library is not automatically installed yet, we need to add its path manually
import sys
sys.path.append("../../")

import numpy as np
import matplotlib.pyplot as plt
import torch
from scipy.spatial import ConvexHull

import implicitmodules.torch as dm

torch.set_default_tensor_type(torch.DoubleTensor)

In [ ]:
data = pickle.load(open("../../data/basipetal.pkl", 'rb'))

Dx = 0.
Dy = 0.
height_source = 38.
height_target = 100.

source = torch.tensor(data['source_d'], dtype=torch.get_default_dtype())
target = torch.tensor(data['target_d'], dtype=torch.get_default_dtype())
source_curve = torch.tensor(data['source_c'], dtype=torch.get_default_dtype())
target_curve = torch.tensor(data['target_c'], dtype=torch.get_default_dtype())

smin, smax = torch.min(source_curve[:, 1]), torch.max(source_curve[:, 1])
sscale = height_source / (smax - smin)
source[:, 1] = Dy - sscale * (source[:, 1] - smax)
source[:, 0] = Dx + sscale * (source[:, 0] - torch.mean(source_curve[:, 0]))
source_curve[:, 1] = Dy - sscale * (source_curve[:, 1] - smax)
source_curve[:, 0] = Dx + sscale * (source_curve[:, 0] - torch.mean(source_curve[:, 0]))

source_curve = source_curve[source_curve[:, 0] <= 0]

hull_fit = ConvexHull(source_curve)
source_curve_convex = source_curve[hull_fit.vertices]

aabb_source = dm.Utilities.AABB.build_from_points(source_curve)

In [ ]:
# pts_implicit1_step = 1.5
# pts_implicit1_x, pts_implicit1_y = torch.meshgrid([
#     torch.arange(0., aabb_source.xmin-pts_implicit1_step, step=-pts_implicit1_step),
#     torch.arange(aabb_source.ymin, aabb_source.ymax+pts_implicit1_step, step=pts_implicit1_step)])

# pts_implicit1 = dm.Utilities.grid2vec(pts_implicit1_x, pts_implicit1_y)
# pts_implicit1_mask = dm.Utilities.is_inside_shape(source_curve_convex, pts_implicit1)
# pts_implicit1 = pts_implicit1[pts_implicit1_mask]

mesh_pts_x, mesh_pts_y = torch.meshgrid([torch.linspace(0, 1, 5), torch.linspace(0, 1, 5)])
pts_implicit1 = dm.Utilities.grid2vec(mesh_pts_x, mesh_pts_y)
source = pts_implicit1.clone()

In [ ]:
# A simple polynomial model of order 1
def pol(pos, a, b, c):
    return a + b*pos[:, 0] + c*pos[:, 1]

In [ ]:
# We create a target
torch.manual_seed(1337)
silent_curve_target = dm.DeformationModules.SilentLandmarks.build_from_points(source_curve)
#silent_curve_target.manifold.fill_cotan(2.*torch.rand_like(source_curve).view(-1).requires_grad_())
silent_dots_target = dm.DeformationModules.SilentLandmarks.build_from_points(source)
#silent_dots_target.manifold.fill_cotan(2.*torch.rand_like(source).view(-1).requires_grad_())

# The model parameters we use and would like to get back
abc_gt = torch.tensor([[1., 0.], [4., 0.], [1., 1.]])

th = 0. * math.pi * torch.ones(pts_implicit1.shape[0])
R_target = torch.stack([dm.Utilities.rot2d(t) for t in th])
C_target = pol(pts_implicit1, abc_gt[0].unsqueeze(1), abc_gt[1].unsqueeze(1), abc_gt[2].unsqueeze(1)).transpose(0, 1).unsqueeze(2)

imp1_pts_cotan = 2.*torch.rand_like(pts_implicit1)
imp1_R_cotan = torch.rand_like(R_target)

implicit1_target = dm.DeformationModules.ImplicitModule1(dm.Manifolds.Stiefel(2, pts_implicit1.shape[0], gd=(pts_implicit1.view(-1).requires_grad_(), R_target.view(-1).requires_grad_()), cotan=(imp1_pts_cotan.clone().view(-1).requires_grad_(), imp1_R_cotan.clone().view(-1).requires_grad_())), C_target, .5, 0.01, 1.)

compound_target = dm.DeformationModules.CompoundModule([silent_curve_target, silent_dots_target, implicit1_target])
dm.HamiltonianDynamic.shoot(dm.HamiltonianDynamic.Hamiltonian(compound_target), 20, "torch_euler")

target_curve = silent_curve_target.manifold.gd.detach().view(-1, 2)
target = silent_dots_target.manifold.gd.detach().view(-1, 2)
target = target + 0.007*torch.randn_like(target)
pts_implicit1_target = implicit1_target.manifold.gd[0].detach().view(-1, 2)

aabb_target = dm.Utilities.AABB.build_from_points(target_curve)

In [ ]:
plt.plot(target.numpy()[:, 0], target.numpy()[:, 1], '.')
plt.plot(source.numpy()[:, 0], source.numpy()[:, 1], 'x')
plt.show()

In [ ]:
implicit1_target1 = dm.DeformationModules.ImplicitModule1(dm.Manifolds.Stiefel(2, pts_implicit1.shape[0], gd=(pts_implicit1.view(-1).requires_grad_(), R_target.view(-1).requires_grad_()), cotan=(imp1_pts_cotan.clone().view(-1).requires_grad_(), imp1_R_cotan.clone().view(-1).requires_grad_())), C_target, .5, 0.01, 1.)

model_target = dm.Models.ModelPointsRegistration(
    [source],
    [implicit1_target1],
    [dm.Attachment.EuclideanPointwiseDistanceAttachement()],
    other_parameters=[abc_gt])

defor, att = model_target.compute([target])

print(defor, att)

In [ ]:
def myParametricModel(modules, fitted_parameters):
    abc = fitted_parameters[-1]
    a = abc[0].unsqueeze(1)
    b = abc[1].unsqueeze(1)
    c = abc[2].unsqueeze(1)
    modules[1]._ImplicitModule1__C = pol(pts_implicit1, a, b, c).transpose(0, 1).unsqueeze(2)

In [ ]:
torch.manual_seed(1337)
sigma1 = 25.
sigma1 = 40.
nu1 = 0.01
coeff1 = 0.5
abc = torch.zeros(3, 2)
abc[0, 0] = 1.
abc[0, 1] = 1.
noise_sigma = 0.2
# abc = abc_gt.clone()
# abc = abc + torch.normal(torch.zeros_like(abc), noise_sigma*torch.ones_like(abc))
print(abc)
print(abc_gt)
C = pol(pts_implicit1, abc[0].unsqueeze(1), abc[1].unsqueeze(1), abc[2].unsqueeze(1)).t().unsqueeze(2)
abc.requires_grad_()
th = 0. * math.pi * torch.ones(pts_implicit1.shape[0])
R = torch.stack([dm.Utilities.rot2d(t) for t in th])

In [ ]:
# Gradcheck
torch.set_printoptions(precision=15)
gradcheck1 = False
gradcheck2 = False
if gradcheck1:
    print("Gradcheck for the first pass (moment fitting)...")
    implicit = dm.DeformationModules.ImplicitModule1(
        dm.Manifolds.Stiefel(2, pts_implicit1.shape[0],
                             gd=(pts_implicit1.view(-1).requires_grad_(),
                                 R.view(-1).requires_grad_()),
                             cotan=(0.1*torch.ones_like(pts_implicit1).view(-1).requires_grad_(),
                                    0.1*torch.ones_like(R).view(-1).requires_grad_())),
        C, sigma1, nu1, coeff1)

    model = dm.Models.ModelPointsRegistration(
        [source],
        [implicit],
        [dm.Attachment.EuclideanPointwiseDistanceAttachement()])

    print(model.gradcheck([target], 1e-2))

if gradcheck2:
    print("Gradcheck for the second pass (polynome fitting)...")
    implicit = dm.DeformationModules.ImplicitModule1(
        dm.Manifolds.Stiefel(2, pts_implicit1.shape[0],
                    gd=(pts_implicit1.view(-1).requires_grad_(),
                        R.view(-1).requires_grad_()),
                    cotan=(0.1*torch.ones(pts_implicit1.shape).view(-1).requires_grad_(),
                           0.1*torch.ones(R.shape).view(-1).requires_grad_())),
        C, sigma1, nu1, coeff1)

    model = dm.Models.ModelPointsRegistration(
        [source],
        [implicit],
        [dm.Attachment.EuclideanPointwiseDistanceAttachement()],
        other_parameters=[abc],
        precompute_callback=myParametricModel,
        fit_moments=False)

    print(model.gradcheck([target], 10.))

In [ ]:
n_steps = 20
n_moments = 10
n_parametric = 10
l = 1000.
weight_curve = 0.1
weight_dots = 10.

mom_source = torch.zeros_like(source).view(-1).requires_grad_()
mom_pts = torch.zeros_like(pts_implicit1).view(-1).requires_grad_()
mom_R = torch.zeros_like(R).view(-1).requires_grad_()

last_C = C
last_abc = abc
last_mom_source = mom_source
last_mom_pts = mom_pts
last_mom_R = mom_R

abcs = []

for i in range(n_steps):
    print("Optimising moments...")
    # We first fit the moments
    implicit1 = dm.DeformationModules.ImplicitModule1(
        dm.Manifolds.Stiefel(2, pts_implicit1.shape[0],
                             gd=(pts_implicit1.view(-1).requires_grad_(),
                                 R.view(-1).requires_grad_()),
                             cotan=(last_mom_pts, last_mom_R)),
        last_C, sigma1, nu1, coeff1)

    model = dm.Models.ModelPointsRegistration(
        [source],
        [implicit1],
        [dm.Attachment.EuclideanPointwiseDistanceAttachement()])

    model._Model__init_manifold[0].fill_cotan(last_mom_source)
    model.compute_parameters()

    fitter = dm.Models.ModelFittingScipy(model, 1., l)
    fitter.fit([target], max_iter=n_moments)

    last_mom_source = model.parameters[0].detach().clone().requires_grad_()
    last_mom_pts = model.parameters[1].detach().clone().requires_grad_()
    last_mom_R = model.parameters[2].detach().clone().requires_grad_()

    # We then try to fit the polynomial model
    print("="*80)
    print("Optimising polynomial model parameters...")
    implicit1_2 = dm.DeformationModules.ImplicitModule1(
        dm.Manifolds.Stiefel(2, pts_implicit1.shape[0],
                    gd=(pts_implicit1.view(-1).requires_grad_(),
                        R.view(-1).requires_grad_()),
                    cotan=(last_mom_pts,
                           last_mom_R)),
        last_C, sigma1, nu1, coeff1)

    model2 = dm.Models.ModelPointsRegistration(
        [source],
        [implicit1_2],
        [dm.Attachment.EuclideanPointwiseDistanceAttachement()],
        other_parameters=[last_abc],
        precompute_callback=myParametricModel,
        fit_moments=False)

    model2._Model__init_manifold[0].fill_cotan(last_mom_source)
    model2.compute_parameters() # TODO: is this necessary?

    fitter2 = dm.Models.ModelFittingScipy(model2, 1., l)
    fitter2.fit([target], max_iter=n_parametric)

    last_abc = model2.parameters[0].detach().clone().requires_grad_()
    abcs.append(last_abc.detach().clone())
    last_C = pol(pts_implicit1, last_abc.detach()[0].unsqueeze(1), last_abc.detach()[1].unsqueeze(1), last_abc.detach()[2].unsqueeze(1)).transpose(0, 1).unsqueeze(2)
    print(last_abc)

In [ ]:
azer = np.zeros([6, len(abcs)])
for i in range(len(abcs)):
    for j in range(6):
        azer[j, i] = abcs[i].view(-1)[j].item()

In [ ]:
for i in range(6):
    plt.subplot(2, 3, i+1)
    plt.plot(range(len(abcs)), azer[i])
    plt.plot([0., len(abcs)], [abc_gt.view(-1)[i]]*2)


plt.show()

In [ ]:
a_ob = abcs[-1][0].detach().unsqueeze(1)
b_ob = abcs[-1][1].detach().unsqueeze(1)
c_ob = abcs[-1][2].detach().unsqueeze(1)
obtained_C = pol(pts_implicit1, a_ob, b_ob, c_ob).transpose(0, 1).unsqueeze(2)

In [ ]:
print(torch.cat([C, obtained_C, C_target], dim=2))

In [ ]:
%matplotlib qt5
from matplotlib.patches import Ellipse
compound = dm.DeformationModules.CompoundModule(model.modules)
compound.manifold.fill(model.init_manifold)
inter = dm.HamiltonianDynamic.shoot(dm.HamiltonianDynamic.Hamiltonian(compound), 20, 'torch_euler')

init = model.init_manifold.copy()

init_dots = init[0].gd.detach().view(-1, 2)
#init_curve = init[1].gd.detach().view(-1, 2)
dots = compound[0].manifold.gd.detach().view(-1, 2)
#curve = compound[1].manifold.gd.detach().view(-1, 2)

plt.axis('equal')
plt.plot(init_dots[:, 0].numpy(), init_dots[:, 1].numpy(), '.k')
#plt.plot(init_curve[:, 0].numpy(), init_curve[:, 1].numpy(), color='black')

plt.plot(dots[:, 0].numpy(), dots[:, 1].numpy(), '.r')
#plt.plot(curve[:, 0].numpy(), curve[:, 1].numpy(), '--r')

plt.plot(target[:, 0].numpy(), target[:, 1].numpy(), 'xb')
#plt.plot(target_curve[:, 0].numpy(), target_curve[:, 1].numpy(), 'b')
plt.show()

In [ ]:

plt.axis('equal')
scale1 = 0.1
scale2 = 0.025
ax1 = plt.subplot(1, 2, 1)
for i in range(pts_implicit1.shape[0]):
    C_i = scale1*C_target[i, :, 0]
    ell = Ellipse(xy=pts_implicit1[i], width=abs(C_i[0]), height=abs(C_i[1]), angle=0.)
    ax1.add_artist(ell)

ax2 = plt.subplot(1, 2, 2)
for i in range(pts_implicit1.shape[0]):
    C_i = scale2*obtained_C[i, :, 0]
    ell = Ellipse(xy=pts_implicit1[i], width=abs(C_i[0]), height=abs(C_i[1]), angle=0.)
    ax2.add_artist(ell)
    

In [ ]:
abcs = [abc.detach().clone()]
def log_abc(model):
    abcs.append(model.parameters[-1].detach().clone())

implicit1 = dm.DeformationModules.ImplicitModule1(
    dm.Manifolds.Stiefel(2, pts_implicit1.shape[0],
                         gd=(pts_implicit1.view(-1).requires_grad_(),
                             R.view(-1).requires_grad_())),
    C_target, 0.5, 0.01, 1.)

#implicit0 = dm.DeformationModules.ImplicitModule0.build_from_points(2, pts_implicit1.shape[0], 0.15, 0.01, coeff=100, gd=pts_implicit1.view(-1).requires_grad_())

model = dm.Models.ModelPointsRegistration(
    [source],
    [implicit1],
    [dm.Attachment.EuclideanPointwiseDistanceAttachement()],
    other_parameters=[abc],
    precompute_callback=myParametricModel)

fitter = dm.Models.ModelFittingScipy(model, 1., 10000., post_iteration_callback=log_abc)

In [ ]:
costs = fitter.fit([target], 150, log_interval=1)

print(model.parameters[-1].detach().numpy())

In [ ]:
print(model.parameters[-1])
print(abc_gt)

In [ ]:
plt.plot(range(len(costs)), costs)
plt.show()